# Advent of Code 2022

[Website](https://adventofcode.com/2022)

## Day 1: Calorie Counting

Find the Elf carrying the most Calories. How many total Calories is that Elf carrying?

In [ ]:
elf = 0
calories_per_elf = [0]

for line in open('01_input.txt', 'r'):
    if line.strip():
        calories_per_elf[elf] += int(line)
    else:
        elf += 1
        calories_per_elf.append(0)

max(calories_per_elf)

Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [ ]:
calories_per_elf.sort(reverse=True)
sum(calories_per_elf[:3])

## Day 2: Rock Paper Scissors

What would your total score be if everything goes exactly according to your strategy guide?

In [ ]:
def get_shape_score(shape):
    return ord(shape) - 87

def get_result_score(opponents_shape, own_shape):
    diff = ord(own_shape) - ord(opponents_shape)
    if diff == 23:
        return 3
    if diff == 21 or diff == 24:
        return 6
    return 0

score = 0
for line in open('02_input.txt', 'r'):
    opponents_shape, own_shape = line.strip().split()
    score += get_shape_score(own_shape)
    score += get_result_score(opponents_shape, own_shape)
print(score)

Following the Elf's instructions for the second column, what would your total score be if everything goes exactly according to your strategy guide?

In [ ]:
def get_own_shape(opponents_shape, result):
    shape_for_loss = { 'A' : 'Z', 'B' : 'X', 'C' : 'Y' }
    shape_for_draw = { 'A' : 'X', 'B' : 'Y', 'C' : 'Z' }
    shape_for_win = { 'A' : 'Y', 'B' : 'Z', 'C' : 'X' }
    
    match result:
        case 'X':
            return shape_for_loss[opponents_shape]
        case 'Y':
            return shape_for_draw[opponents_shape]
        case _:
            return shape_for_win[opponents_shape]

score = 0
for line in open('02_input.txt', 'r'):
    opponents_shape, result = line.strip().split()
    own_shape = get_own_shape(opponents_shape, result)
    score += get_shape_score(own_shape)
    score += get_result_score(opponents_shape, own_shape)
print(score)

## Day 3: Rucksack Reorganization

Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?

In [ ]:
def to_priority(item):
    if(ord(item) < 91):
        return ord(item) - 38
    return ord(item) - 96

priorities = 0
for line in open('03_input.txt', 'r'):
    compartment_len = len(line)//2
    compartment0, compartment1 = line[:compartment_len], line[compartment_len:]
    duplicate_items = set(compartment0).intersection(compartment1)
    assert len(duplicate_items) == 1
    priorities += to_priority(duplicate_items.pop())
print(priorities)

Find the item type that corresponds to the badges of each three-Elf group. What is the sum of the priorities of those item types?

In [ ]:
lines = open('03_input.txt', 'r').readlines()
lines = [line.strip() for line in lines]
assert len(lines) % 3 == 0

priorities = 0
group_idx = 0
while group_idx < len(lines):
    groups = lines[group_idx:group_idx+3]
    duplicate_items = set(groups[0]).intersection(groups[1]).intersection(groups[2])
    assert len(duplicate_items) == 1
    priorities += to_priority(duplicate_items.pop())
    group_idx += 3
print(priorities)

## Day 4: Camp Cleanup

In how many assignment pairs does one range fully contain the other?

In [ ]:
def get_assignments(line):
    assignment0, assignment1 = line.strip().split(',')
    assignment0 = [int(a) for a in assignment0.split('-')]
    assignment1 = [int(a) for a in assignment1.split('-')]
    return (assignment0, assignment1)

def is_fully_contained(assignments):
    return (assignments[0][0] <= assignments[1][0] and assignments[0][1] >= assignments[1][1]) or \
           (assignments[0][1] <= assignments[1][1] and assignments[0][0] >= assignments[1][0])

count = 0
for line in open('04_input.txt', 'r'):
    assignments = get_assignments(line)
    count += is_fully_contained(assignments)
print(count)

In how many assignment pairs do the ranges overlap?

In [ ]:
def is_overlapping(assignments):
    return (assignments[0][0] <= assignments[1][1] and assignments[0][0] >= assignments[1][0]) or \
           (assignments[0][1] <= assignments[1][1] and assignments[0][1] >= assignments[1][0]) or \
           (assignments[1][0] <= assignments[0][1] and assignments[1][0] >= assignments[0][0]) or \
           (assignments[1][1] <= assignments[0][1] and assignments[1][1] >= assignments[0][0])

count = 0
for line in open('04_input.txt', 'r'):
    assignments = get_assignments(line)
    count += is_overlapping(assignments)
print(count)

## Day 5: Supply Stacks

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [ ]:
import re

def parse_input(lines):
    def simplify(stacks_str):
        result = []
        for line in stacks_str:
            while '  ' in line:
                line = line.replace(']    ', '] [.]')
            result.append(line.replace('] [', '').replace('[', '').replace(']\n', ''))
        return result

    def parse_stacks(stacks_str):
        stacks_str = simplify(stacks_str)

        num_stacks = max(len(line) for line in stacks_str)
        stacks = [[] for _ in range(num_stacks)]

        stacks_str.reverse()
        for line in stacks_str:
            for stack_id in range(len(line)):
                if(line[stack_id] == '.'):
                    continue
                stacks[stack_id].append(line[stack_id])
        return stacks
    
    def parse_operations(operations_str):
        operations = []
        for line in operations_str:
            operations.append({
                'quantity': int(re.search('move (.*) from', line).group(1)),
                'source': int(re.search('from (.*) to', line).group(1)) - 1,
                'target': int(re.search('to (.*)\n', line).group(1)) - 1
                })
        return operations

    legend_line_idx = lines.index(next(filter(lambda line : line.startswith(' 1'), lines)))
    stacks_str = lines[:legend_line_idx]
    operations_str = lines[legend_line_idx+2:]
    return parse_stacks(stacks_str), parse_operations(operations_str)

def process_operation(stacks, operation):
    for _ in range(operation['quantity']):
        crate = stacks[operation['source']].pop()
        stacks[operation['target']].append(crate)
    return stacks

def process(stacks, operations):
    for operation in operations:
        stacks = process_operation(stacks, operation)
    return stacks


lines = open('05_input.txt', 'r').readlines()
stacks, operations = parse_input(lines)
updated_stacks = process(stacks, operations)

for stack in updated_stacks:
    print(stack.pop(), end='')

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [ ]:
def process_operation(stacks, operation):
    crates = [stacks[operation['source']].pop() for _ in range(operation['quantity'])]
    crates.reverse()
    [stacks[operation['target']].append(crate) for crate in crates]
    return stacks

lines = open('05_input.txt', 'r').readlines()
stacks, operations = parse_input(lines)
updated_stacks = process(stacks, operations)

for stack in updated_stacks:
    print(stack.pop(), end='')

## Day 6: Tuning Trouble

How many characters need to be processed before the first start-of-packet marker is detected?

In [ ]:
def is_unique_chars(string):
    return len(set(string)) == len(list(string))

def get_marker_end_position(message, marker_len):
    for start_idx in range(len(message)-marker_len):
        marker = message[start_idx:start_idx+marker_len]
        if(is_unique_chars(marker)):
            return start_idx + marker_len
    raise Exception('No marker of length ' + str(marker_len) + ' found.')

line = open('06_input.txt', 'r').readline()
get_marker_end_position(line, 4)

How many characters need to be processed before the first start-of-message marker is detected?

In [ ]:
line = open('06_input.txt', 'r').readline()
get_marker_end_position(line, 14)

## Day 7: No Space Left On Device

What is the sum of the total sizes of those directories?

In [ ]:
class Directory:
    subdirs: dict
    "key: name, value: directory"

    files: dict
    "key: name, value: directory"

    total_size: int
    "total size including all subdirs"

    def __init__(self, name, parent = None):
        self.name = name
        self.parent = parent
        self.subdirs = {}
        self.files = {}
        self.total_size = None

def parse_input(lines):
    root = Directory('/')
    current_dir = root

    for line in lines:
        match line.removeprefix('$').split():
            case['dir', name]:
                current_dir.subdirs[name] = Directory(name, current_dir)
            case['cd', target]:
                if target == '/':
                    current_dir = root
                elif target == '..':
                    current_dir = current_dir.parent
                else:
                    current_dir = current_dir.subdirs[target]
            case[size, name] if size.isnumeric():
                current_dir.files[name] = int(size)
    return root

def add_total_dir_sizes(current_dir):
    size = sum(current_dir.files.values())
    for subdir in current_dir.subdirs.values():
        size += add_total_dir_sizes(subdir)
    current_dir.total_size = size
    return size

def calculate_size_sum(current_dir, upper_threshold):
    sum = 0
    if current_dir.total_size < upper_threshold:
        sum += current_dir.total_size
    for subdir in current_dir.subdirs.values():
        sum += calculate_size_sum(subdir, upper_threshold)
    return sum

lines = open('07_input.txt', 'r').readlines()
lines = [line.strip() for line in lines]

root = parse_input(lines)
add_total_dir_sizes(root)
calculate_size_sum(root, 100_000)

What is the total size of that directory?

In [ ]:
from sys import maxsize

free_space = 70_000_000 - root.total_size
space_to_be_freed = 30_000_000 - free_space

def get_cleanup_size(current_dir, space_to_be_freed):
    best_match = maxsize
    if current_dir.total_size >= space_to_be_freed:
        best_match = min(best_match, current_dir.total_size)
    for subdir in current_dir.subdirs.values():
        best_match = min(best_match, get_cleanup_size(subdir, space_to_be_freed))
    return best_match

get_cleanup_size(root, space_to_be_freed)

## Day 8: Treetop Tree House

Consider your map; how many trees are visible from outside the grid?

In [ ]:
import numpy as np

heights = np.genfromtxt('08_input.txt', delimiter=1)

visibilities = np.ones_like(heights)
for idx, height in np.ndenumerate(heights[1:-1,1:-1]):
    row_idx = idx[0] + 1
    col_idx = idx[1] + 1
    max_height_above = np.amax(heights[:row_idx, col_idx])
    max_height_below = np.amax(heights[row_idx+1:, col_idx])
    max_height_right = np.amax(heights[row_idx,:col_idx])
    max_height_left = np.amax(heights[row_idx,col_idx+1:])
    visibilities[row_idx, col_idx] = any(m < height for m in [max_height_above, max_height_below, max_height_right, max_height_left])

np.count_nonzero(visibilities)

What is the highest scenic score possible for any tree?

In [ ]:
def get_viewing_distance(height, neighbour_heights):
    distance = 0
    for neighbour_height in neighbour_heights:
        distance += 1
        if(neighbour_height >= height):
            break
    return distance

scenic_scores = np.ones_like(heights)
for idx, height in np.ndenumerate(heights):
    row_idx = idx[0]
    col_idx = idx[1]
    scenic_scores[idx] *= get_viewing_distance(height, heights[:row_idx, col_idx][::-1]) #look up
    scenic_scores[idx] *= get_viewing_distance(height, heights[row_idx+1:, col_idx]) #look down
    scenic_scores[idx] *= get_viewing_distance(height, heights[row_idx,col_idx+1:]) #look right
    scenic_scores[idx] *= get_viewing_distance(height, heights[row_idx,:col_idx][::-1]) #look left

np.max(scenic_scores)